<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/XGB_SVM(SGD)_Logistc_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!pip install bert-for-tf2

     |████████████████████████████████| 1.2MB 4.3MB/s 
     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=1776b3404539fdb3afe5b7afa66bb0e2cfe36e981c7816804fb77e0bf47794b6
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=69138135989ea2c2977a484b827f6732668978b9f354878e70fc52e62e89bdcc
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=e7bdcae2fa0e0d478afa5eca74675119d99ee6367cb869eaf8b67d604f55561a
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re, bert, tokenization
import tensorflow_hub as hub
import tensorflow as tf


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

 96% 111M/115M [00:01<00:00, 96.5MB/s]
100% 115M/115M [00:01<00:00, 91.4MB/s]


In [4]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [5]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
259200,college debt with scholarship vs without schol...,teenagers
106830,I need a late night friend I think. This balon...,depression
214848,SchoolworkIs anyone else just so occupied with...,SuicideWatch
130874,Singing the theme song to The Fresh Prince of ...,depression
83036,dude id hate to be a mod... just doing your jo...,teenagers


In [6]:
data.shape

(348111, 2)

In [7]:
def preprocess(string):
    phrase = str(string)
    phrase = re.sub('[^a-z]+', ' ', phrase, flags = re.IGNORECASE)
    phrase = re.sub('(\s+)', ' ', phrase)
    phrase = phrase.lower()
    # words_li = ['suicide', 'kill', 'depression', 'fuck', 'filler', 'die', 'depressed']
    li = list(stopwords.words()) #+ words_li
    text_tokens = word_tokenize(phrase)
    return " ".join([word for word in text_tokens if word not in li])
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [ ]:
data['text'] = data['text'].map(lambda string: preprocess(string))

In [ ]:
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [22]:
X = data.iloc[:, 0].values
y = data.iloc[:, 1].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.8, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.3, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(83546,) (69623,) (194942,)


In [9]:
xgb1 = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])

xgb1.fit(train_X, train_y)
y_pred = xgb1.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_y))
print(classification_report(test_y, y_pred))

accuracy 0.7400140758082817
              precision    recall  f1-score   support

           0       0.72      0.65      0.69     23207
           1       0.73      0.66      0.69     23208
           2       0.77      0.91      0.83     23208

    accuracy                           0.74     69623
   macro avg       0.74      0.74      0.74     69623
weighted avg       0.74      0.74      0.74     69623



In [10]:
from sklearn.linear_model import SGDClassifier

sgd2 = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier()),
              ])

sgd2.fit(train_X, train_y)
y_pred = sgd2.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_y))
print(classification_report(test_y, y_pred))

accuracy 0.796389124283642
              precision    recall  f1-score   support

           0       0.76      0.74      0.75     23207
           1       0.77      0.71      0.74     23208
           2       0.84      0.93      0.89     23208

    accuracy                           0.80     69623
   macro avg       0.79      0.80      0.79     69623
weighted avg       0.79      0.80      0.79     69623



In [11]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-04-19 12:59:39--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.75.62
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.75.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  69.4MB/s    in 23s     

2021-04-19 13:00:02 (67.6 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [15]:
import gensim
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("/content/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [16]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [17]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
for i in range(train_X.shape[0]):
    train_X[i] = w2v_tokenize_text(train_X[i])

for i in range(test_X.shape[0]):
    test_X[i] = w2v_tokenize_text(test_X[i])


In [18]:
X_train_word_average = word_averaging_list(wv,train_X)
X_test_word_average = word_averaging_list(wv,test_X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  


In [19]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, solver = 'newton-cg')
logreg = logreg.fit(X_train_word_average, train_y)
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test_y))
print(classification_report(test_y, y_pred))

accuracy 0.7603665455381124
              precision    recall  f1-score   support

           0       0.72      0.68      0.69     23208
           1       0.69      0.75      0.72     23208
           2       0.88      0.86      0.87     23207

    accuracy                           0.76     69623
   macro avg       0.76      0.76      0.76     69623
weighted avg       0.76      0.76      0.76     69623



In [35]:
import itertools


%matplotlib inline


from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

max_words = 750
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_X) # only fit on train

x_train = tokenize.texts_to_matrix(train_X)
x_test = tokenize.texts_to_matrix(test_X)

num_classes = 3
y_train = utils.to_categorical(train_y, num_classes)
y_test = utils.to_categorical(test_y, num_classes)

batch_size = 32
epochs = 10

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [checkpoint, earlystopping],
                    validation_split=0.1)


Epoch 1/10
2350/2350 [==============================] - 13s 5ms/step - loss: 0.6424 - accuracy: 0.7253 - val_loss: 0.5312 - val_accuracy: 0.7791

Epoch 00001: val_accuracy improved from -inf to 0.77905, saving model to model.h5
Epoch 2/10
2350/2350 [==============================] - 12s 5ms/step - loss: 0.5070 - accuracy: 0.7874 - val_loss: 0.5239 - val_accuracy: 0.7843

Epoch 00002: val_accuracy improved from 0.77905 to 0.78432, saving model to model.h5
Epoch 3/10
2350/2350 [==============================] - 12s 5ms/step - loss: 0.4671 - accuracy: 0.8053 - val_loss: 0.5196 - val_accuracy: 0.7820

Epoch 00003: val_accuracy did not improve from 0.78432
Epoch 4/10
2350/2350 [==============================] - 13s 5ms/step - loss: 0.4330 - accuracy: 0.8195 - val_loss: 0.5396 - val_accuracy: 0.7750

Epoch 00004: val_accuracy did not improve from 0.78432
Epoch 5/10
2350/2350 [==============================] - 13s 5ms/step - loss: 0.3862 - accuracy: 0.8432 - val_loss: 0.5459 - val_accuracy: 0

In [36]:
y_pred = model.predict(x_test)

In [37]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.7750456027462189
              precision    recall  f1-score   support

           0       0.74      0.71      0.72     24088
           1       0.70      0.75      0.72     21763
           2       0.89      0.87      0.88     23772

    accuracy                           0.78     69623
   macro avg       0.78      0.77      0.77     69623
weighted avg       0.78      0.78      0.78     69623

